In [3]:
from lxml import html
import requests
from collections import OrderedDict
import json
import argparse
import pandas as pd
import csv

def parse(input_file):
    restaurants = []
    for line in input_file:
        url = line
        response = requests.get(url).text
        parser = html.fromstring(response)
	
        XPATH_RATING = '//div[@id="ratingFilter"]//ul//li'

        XPATH_TRAVELER = '//div[@class="col segment extraWidth"]//ul//li'
        XPATH_TIME = '//div[@class="col season extraWidth"]//ul/li'
        XPATH_LANGUAGE = '//div[@class="col language extraWidth"]//ul/li'
#        XPATH_DETAILS = '//div[contains(@class,"details_tab")]//div[contains(@class, "table_section")]//text()'
        XPATH_NAME = '//h1[@property="name"]//text()'
        XPATH_HOTEL_RATING = '//span[@property="ratingValue"]//@content'
        XPATH_REVIEWS = '//a[@property="reviewCount"]/@content'
        XPATH_RANK = '//div[@class="slim_ranking"]//text()'
        XPATH_OFFICIAL_DESCRIPTION = '//div[contains(@class,"additional_info")]//span[contains(@class,"tabs_descriptive_text")]//text()'
        XPATH_LOCATION = '//div[@class="mapContainer"]//text()'
                                        
        ratings = parser.xpath(XPATH_RATING)
        travelers = parser.xpath(XPATH_TRAVELER)
        times = parser.xpath(XPATH_TIME)
        languages = parser.xpath(XPATH_LANGUAGE)
	
        raw_name = parser.xpath(XPATH_NAME)
        raw_rank = parser.xpath(XPATH_RANK)
        raw_review_count = parser.xpath(XPATH_REVIEWS)
        raw_rating = parser.xpath(XPATH_HOTEL_RATING)
        raw_official_description = parser.xpath(XPATH_OFFICIAL_DESCRIPTION)
#        raw_details = parser.xpath(XPATH_DETAILS)
        raw_location = parser.xpath(XPATH_LOCATION)
					
        name = ''.join(raw_name).strip() if raw_name else None
        rank = ''.join(raw_rank).strip() if raw_rank else None
        review_count = ''.join(raw_review_count).strip() if raw_review_count else None
        hotel_rating = ''.join(raw_rating).strip() if raw_rating else None
        official_description = ' '.join(' '.join(raw_official_description).split()) if raw_official_description else None
#        details = ' '.join(' '.join(raw_details).split()) if raw_details else None
        location = ' '.join(' '.join(raw_location).split()) if raw_location else None       

        ratings_dict = OrderedDict()
        for rating in ratings:
            XPATH_RATING_KEY = './/div[@class="row_label"]//text()'
            XPATH_RATING_VALUE = './/span[@class="row_bar"]/following-sibling::span//text()'
            raw_rating_key = rating.xpath(XPATH_RATING_KEY)
            raw_rating_value = rating.xpath(XPATH_RATING_VALUE)
            cleaned_rating_key = ''.join(raw_rating_key).replace('\n','')
            cleaned_rating_value = ''.join(raw_rating_value).replace('\n','')
            ratings_dict.update({cleaned_rating_key:cleaned_rating_value})
    
        travelers_list = [] 
        for traveler in travelers:
            XPATH_TRAVELER_TYPE = './/label//text()'
            XPATH_TRAVELER_COUNT = './/span//text()'
            raw_traveler_type = traveler.xpath(XPATH_TRAVELER_TYPE)
            raw_traveler_count = traveler.xpath(XPATH_TRAVELER_COUNT)
            cleaned_traveler_type = ''.join(raw_traveler_type).replace('\n','').split(' ')
            cleaned_traveler_type = cleaned_traveler_type[0]
            cleaned_traveler_count = ''.join(raw_traveler_count).replace('\n','')
            travelers_list.append((cleaned_traveler_type,cleaned_traveler_count))
        
        times_list = []
        for time in times:
            XPATH_TIME_TYPE = './/label//text()'
            XPATH_TIME_COUNT = './/span//text()'
            raw_time_type = time.xpath(XPATH_TIME_TYPE)
            raw_time_count = time.xpath(XPATH_TIME_COUNT)
            cleaned_time_type = ''.join(raw_time_type).replace('\n','').split(' ')
            cleaned_time_type = cleaned_time_type[0]
            cleaned_time_count = ''.join(raw_time_count).replace('\n','')
            times_list.append((cleaned_time_type, cleaned_time_count))
        
        language_list = []
        for language in languages:
            XPATH_LANGUAGE_TYPE = './/label//text()'
#            XPATH_LANGUAGE_COUNT = './/span//text()'
            raw_language_type = language.xpath(XPATH_LANGUAGE_TYPE)
#            raw_language_count = language.xpath(XPATH_LANGUAGE_COUNT)
            cleaned_language_type = ''.join(raw_language_type).replace('\n','').split(' ')
            cleaned_language_type = cleaned_language_type[0]
#            cleaned_language_count = ''.join(raw_language_count).replace('\n','')
#            languages_dict.update({cleaned_language_type:cleaned_language_count})
            language_list.append(cleaned_language_type)
        language_list = language_list[1:]


        data = {'name':name,
				'rank':rank,
				'rating':hotel_rating,
                'review_count':review_count,
                'location':location,
                'official_description':official_description,
                  'travelers':travelers_list,
                  'times of year':times_list,
                  'languages':language_list
#                  'details':details
	    }
        restaurants.append(data)
        
        
#    columns = ['property_title', 'rank', 'rating', 'review_count', 'location', 'official_description', 'travelers', 'times', 'langauges']
    df = pd.DataFrame(restaurants)
    return df



input_file = open('restaurants.txt', 'r')
scraped_data = parse(input_file)
scraped_data

,languages,location,name,official_description,rank,rating,review_count,times of year,travelers
0,"[English(88), German(3), Italian(3), ]",,Turtle Green Tea Bar,None,#3 of 29 Coffee & Tea in Amman \n#61 of 593 Pl...,4.5,103,"[(Mar-May, (31)), (Jun-Aug, (24)), (Sep-Nov, (...","[(Families, (6)), (Couples, (9)), (Solo, (16))..."
1,"[English(10), French(1)]",,Mr. Lollies,None,#3 of 25 Dessert in Amman \n#95 of 593 Places ...,5,11,"[(Mar-May, (3)), (Jun-Aug, (2)), (Sep-Nov, (5)...","[(Families, (1)), (Couples, (1)), (Solo, (0)),..."
2,[English(2)],None,Al-Raqaq Saj,None,#229 of 545 Restaurants in Amman,5,2,"[(Mar-May, (1)), (Jun-Aug, (1)), (Sep-Nov, (0)...","[(Families, (2)), (Couples, (0)), (Solo, (0)),..."
3,"[English(45), Arabic(5), Spanish(3), ]",,Mijana Restaurant,None,#45 of 545 Restaurants in Amman,4.5,56,"[(Mar-May, (12)), (Jun-Aug, (15)), (Sep-Nov, (...","[(Families, (5)), (Couples, (10)), (Solo, (1))..."
4,"[English(43), Italian(5), French(2), ]",,Zorba Touristic Restaurant,None,#148 of 545 Restaurants in Amman,4,53,"[(Mar-May, (11)), (Jun-Aug, (16)), (Sep-Nov, (...","[(Families, (8)), (Couples, (4)), (Solo, (1)),..."
5,"[English(49), Portuguese(2), Arabic(1), ]",,Caffe Strada,None,#39 of 545 Restaurants in Amman,4.5,53,"[(Mar-May, (9)), (Jun-Aug, (9)), (Sep-Nov, (11...","[(Families, (2)), (Couples, (9)), (Solo, (11))..."
6,"[English(74), Arabic(3), Spanish(2)]",,Firefly Burger,None,#69 of 545 Restaurants in Amman,4,79,"[(Mar-May, (21)), (Jun-Aug, (18)), (Sep-Nov, (...","[(Families, (13)), (Couples, (10)), (Solo, (4)..."
7,"[English(60), Arabic(3), German(1), ]",,Tche Tche,None,#373 of 545 Restaurants in Amman,3,67,"[(Mar-May, (15)), (Jun-Aug, (18)), (Sep-Nov, (...","[(Families, (11)), (Couples, (5)), (Solo, (5))..."
8,[English(3)],,Mr. Krepe,None,#462 of 545 Restaurants in Amman,2.5,3,"[(Mar-May, (0)), (Jun-Aug, (0)), (Sep-Nov, (1)...","[(Families, (0)), (Couples, (1)), (Solo, (0)),..."
9,"[English(170), Italian(7), French(5), ]",,Al-Quds,None,#12 of 545 Restaurants in Amman,4.5,197,"[(Mar-May, (47)), (Jun-Aug, (38)), (Sep-Nov, (...","[(Families, (30)), (Couples, (33)), (Solo, (22..."


In [4]:
with open('restaurants_details.csv', 'w') as csvfile:
    scraped_data.to_csv(csvfile, sep='\t', encoding='utf-8')